In [1]:
import pandas as pd
import requests
import urllib.parse
import urllib.request
import sqlite3
from datetime import datetime
from urllib.request import urlopen
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from tqdm import tqdm
# define the header
# write your own header here
# get your header at "chrome://version"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
# create engine
engine = create_engine('sqlite:///../../dataset/us/us_ticker_revenue_mac.db')
# # create database connection
# database = '../../dataset/us/us_ticker_eps_annual.db'
# conn = sqlite3.connect(database)
# print('sql connect success!')
# import the list
name_engine = create_engine('sqlite:///../../dataset/us/us_ticker_list_with_name.db')
log_engine = create_engine('sqlite:///../../dataset/us/us_log_record.db')
name_df = pd.read_sql('TOTAL', con=name_engine)

In [2]:
runtime = datetime.now()
if runtime.month < 10:
    if runtime.day < 10:
        runtime = str(runtime.year) + '0' + str(runtime.month) + '0' +str(runtime.day)
    else:
        runtime = str(runtime.year) + '0' + str(runtime.month) + str(runtime.day)
else:
    runtime = str(runtime.year) + str(runtime.month) + str(runtime.day)

In [3]:
ticker_symbol_list = name_df.Symbol.to_list()
ticker_name_list = name_df['Company Name'].to_list()
ticker_length = len(ticker_symbol_list)

In [4]:
for i in tqdm(range(ticker_length)):
        try:
                html_data = requests.get('https://www.macrotrends.net/stocks/charts/{}/{}/revenue'.format(ticker_symbol_list[i], ticker_name_list[i]),headers=headers, timeout=20)
                soup = BeautifulSoup(html_data.text, 'lxml')
                target_table = soup.find_all('table', attrs={'class':'historical_data_table'})
                # Annual
                target_table_01 = target_table[0]
                ticker_revenue_annual = pd.DataFrame(columns=['datetime','revenue'])
                log_df = pd.DataFrame(columns=['symbol','status'])
                # ------------------------------- # 
                #        get annual revenue
                # ------------------------------- #
                for row in target_table_01.find_all('tr'):
                        col = row.find_all('td')
                        len_col = len(col)
                        if len_col == 2:
                                date = col[0].text
                                revenue = col[1].text[1:].replace(',','')
                                temp = pd.DataFrame([[date, revenue]],columns=['datetime','revenue'])
                                ticker_revenue_annual = pd.concat([ticker_revenue_annual,temp],ignore_index=True)
                ticker_revenue_annual.dropna(inplace=True)
                ticker_revenue_annual.revenue = ticker_revenue_annual.revenue.astype(int)
                ticker_revenue_annual.to_sql('{}-Annual'.format(ticker_symbol_list[i]), con=engine, if_exists='replace',index=None)

                # ------------------------------- # 
                #        get quarter revenue
                # ------------------------------- #
                # Quarter 
                target_table_02 = target_table[1]
                ticker_revenue_quarter = pd.DataFrame(columns=['datetime','revenue'])
                for row in target_table_02.find_all('tr'):
                        col = row.find_all('td')
                        len_col = len(col)
                        if len_col == 2:
                                date = col[0].text
                                revenue = col[1].text[1:].replace(',','')
                                temp = pd.DataFrame([[date, revenue]],columns=['datetime','revenue'])
                                ticker_revenue_quarter = pd.concat([ticker_revenue_quarter,temp],ignore_index=True)
                ticker_revenue_quarter.revenue = ticker_revenue_quarter.revenue.astype(float)
                ticker_revenue_quarter.to_sql('{}-Quarter'.format(ticker_symbol_list[i]),con=engine, if_exists='replace',index=None)
        except:
                status = 'fail'
                temp_log = pd.DataFrame([[ticker_symbol_list[i], status]], columns=['symbol', 'status'])
                log_df = pd.concat([log_df,temp_log])
                log_df.to_sql('{}-revenue-record'.format(time),con=log_engine,if_exists='replace',index=None)

100%|██████████| 6247/6247 [12:38:48<00:00,  7.29s/it]


In [27]:
target_table_01

<table class="historical_data_table table">
<thead>
<tr>
<th colspan="2" style="text-align:center">Microsoft Annual Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:center">2022</td>
<td style="text-align:center">$198,270</td>
</tr>
<tr>
<td style="text-align:center">2021</td>
<td style="text-align:center">$168,088</td>
</tr>
<tr>
<td style="text-align:center">2020</td>
<td style="text-align:center">$143,015</td>
</tr>
<tr>
<td style="text-align:center">2019</td>
<td style="text-align:center">$125,843</td>
</tr>
<tr>
<td style="text-align:center">2018</td>
<td style="text-align:center">$110,360</td>
</tr>
<tr>
<td style="text-align:center">2017</td>
<td style="text-align:center">$96,571</td>
</tr>
<tr>
<td style="text-align:center">2016</td>
<td style="text-align:center">$91,154</td>
</tr>
<tr>
<td style="text-align:center">2015</td>
<td style="text-align:center">$93,580</td>
</tr>
<tr>
<td style="text-align:cent

In [28]:
for row in target_table_01.find_all('tr'):
        col = row.find_all('td')
        len_col = len(col)
        if len_col == 2:
                date = col[0].text
                revenue = col[1].text[1:].replace(',','')
                temp = pd.DataFrame([[date, revenue]],columns=['datetime','revenue'])
                ticker_revenue_annual = pd.concat([ticker_revenue_annual,temp],ignore_index=True)

In [29]:
ticker_revenue_annual

,datetime,revenue
0,2022,198270
1,2021,168088
2,2020,143015
3,2019,125843
4,2018,110360
5,2017,96571
6,2016,91154
7,2015,93580
8,2014,86833
9,2013,77849


In [13]:
num1 = '$51,865'
num1 = num1[1:].replace(',','')
num1 = int(num1)
num1

51865